# **Introduction to 1D phonons**

<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/master/notebook/statistical-mechanics/ising_model.ipynb

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Tasks and exercises**

1. Visualize the lattice vibration at $k=0$ and $k=\frac{\pi}{a}$, what characteristics do you observe ? 
    <details>
    <summary style="color: red">Solution</summary>
    The atom indexed $n$ has a displacement given by $u=\exp(i(n\cdot ka-\omega t))$.
    
    At $k=0$, the displacement is given by $u=\exp(-i\omega t)$. Therefore, all atoms are displaced identically, which creates a translation. However, since $\omega=0$ at $k=0$, atoms stay static.
    
    At $k=\frac{\pi}{a}$, the displacement is given by $u=\exp(i(n\cdot\pi-\omega t))=\exp(i\cdot n\cdot\pi)\exp(-i\omega t)$. Therefore, the neighbouring atom at $n+1$ has an amplitude given by $u'=u\exp(i\pi)=-1$. Thus, neighbouring atoms oscillate out-of-phase.
    
    To make atoms oscillate out-of phase, a lot of energy is required since the phonon contains a lot of potential energy. As energy is directly linked to frequency with $E=h\nu$, the maximum of frequency is thus found at $k=\pm \frac{\pi}{a}$.
    </details>


2. The first Brillouin zone is defined between $\frac{-\pi}{a}$ and $\frac{\pi}{a}$. Explain why the first Brillouin zone contains all the necessary information by looking at $k'=k+\frac{2\pi}{a}$
    <details>
    <summary style="color: red">Solution</summary>
    At $k'=k+2\frac{\pi}{a}$ the lattice vibration stays identical to the phonon with wave vector $k$. Therefore, the frequency will also be identical.
    
    From the equation of displacement, we get $u'=\exp(i(n\cdot k'a-\omega t))=\exp(i(n\cdot ka-\omega t))\exp(i\cdot n\cdot2\pi)
    =u$.

    </details>


3. Visualize the phonon dispersion for a diatomic chain. What is the difference between the acoustic and optical branch at $k=0$ and $k=\pm\frac{\pi}{2a}$ ? How do the atoms displacements relate to frequency ?
    <details>
    <summary style="color: red">Solution</summary>
    At $k=0$, in the acoustic branch, as in the monoatomic chain, the atoms do not move. Instead, in the optical branch the two atoms are moving out-of-phase. The latter one has therefore a high frequency.
    
    At $k=\pm\frac{\pi}{2a}$, in the acoustic branch the light atoms stand still whearas the heavier atoms move. In the optical branch, the opposite is observed. Since frequency is proportional to $\sqrt{1/M}$, the frequency is higher when lighter atoms move. 
    </details>
    
    
4. Adjust the mass ratio of the molecules. What happens when the ratio is high ? When the ratio is close to unity ? How does the band dispersion evolve ?
    <details>
    <summary style="color: red">Solution</summary>
    As the ratio grows, the forbidden frequencies gap grows as well. At $k=\pm\frac{\pi}{2a}$, the gap is proportional to $\sqrt{1/M_{gray}}-\sqrt{1/M_{red}}$.
    
    Furthermore, at $k=0$, we have $\frac{u_{red}}{u_{gray}}=\frac{-M_{gray}}{M_{red}}$, and thus the heavier atom amplitude decreases as it gets heavier.
    </details>
    
    
5. Set the mass ratio to 1. How does it compare to the case of the monoatomic chain ?
    <details>
    <summary style="color: red">Solution</summary>
    When the ratio is 1, the chain is identical to the monoatomic one, except that the lattice parameter is twice as big, shrinking the Brillouin zone to half the size of the monoatomic case. One recognizes the dispersion relation of the monoatomic chain in the first BZ between $k=-\frac{\pi}{a}$ and $k=\frac{\pi}{a}$.
    
    The optical branch in the first BZ now represents the monoatomic dispersion relation but at $k'=k\pm\frac{\pi}{a}$. This is called band folding.
    </details>
   

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
import matplotlib
matplotlib.rcParams['figure.facecolor']='w'
from ase.io.trajectory import Trajectory
from ase import Atoms

from NGLTrajectoryClass import NGLTrajectory

import ipywidgets as widgets
from ipywidgets import jslink,HBox, VBox, HTMLMath,Label,IntSlider, Tab,Play, Layout
from IPython.display import display

In [ ]:
traj=Trajectory('dummy.traj','w')
traj.write(Atoms(2*'C',[[0,0,0],[1,1,1]]))
handler=NGLTrajectory(trajectory=Trajectory('dummy.traj','r'))
handler.view.stage.set_parameters(mouse_preset="pymol")

handler.view._js("""
this.mouseOverDisplay('block')
var container = this.stage.viewer.container
var that = this
container.addEventListener('mouseout', function(e) {
          that.mouseOverDisplay('block')
      }, false);
""")

widgets=[
    handler.slider_C,
    handler.slider_M,
    handler.slider_amplitude,
    handler.button_chain,
]
for widget in widgets:
    handler.widgetList.append(widget)

handler.slider_atom_radius.value=0.1
handler.slider_arrow_radius.value=0.1
handler.tick_box_arrows.value=False
handler.set_player_parameters(delay=handler.init_delay)
handler.set_view_parameters(clipDist=1,quality='low')

handler.initialize_dispersion_plot()
handler.button_chain.value='diatomic'
handler.compute_dispersion()
handler.compute_dispersion_relation()
handler.button_chain.value='monoatomic'
handler.compute_dispersion()
handler.compute_dispersion_relation()

handler.compute_trajectory_1D()
handler.band_dispersion()

handler.set_view_dimensions()
handler.view.stage.set_parameters(mouse_preset="pymol")


chain= HBox([handler.button_chain_description,handler.button_chain])
mass_ratio=handler.output_ratio
click=HTMLMath(value='''Click on the plot to get the corresponding phonon''')
phonon = HBox(
    [VBox([chain,mass_ratio]), VBox([click,handler.fig.canvas])]
)

camera_orientation=HBox([handler.camera_orientation_description,handler.output_camera_position])
camera_text_orientation=handler.text_orientation
camera_error=handler.output_camera_position_error
camera_position=VBox([camera_orientation,camera_text_orientation,camera_error])

amplitude=HBox([handler.slider_amplitude_description,handler.slider_amplitude])
radius=HBox([handler.slider_atom_radius_description,handler.slider_atom_radius])
atom=VBox([amplitude,radius])

appearance=HBox([handler.arrow,atom])
gif=HBox([handler.movie,handler.output_gif])


tab = Tab()
tab.children = [phonon, appearance, gif,camera_position, HBox()]
titles = ["Phonon", "Appearance", "Generate GIF", "Camera settings", "Hide parameters"]
for i in range(5):
    tab.set_title(i, titles[i])
    

camera_actions_1=HTMLMath(value='''Hover to play animation<br> 
Scroll to zoom''')
camera_actions_2=HTMLMath(value='''
Left click to rotate<br> 
Right click to translate 
''')
spacer=HTMLMath(value='''<p style="color: white"></p>''')

helper_text=HBox([spacer,camera_actions_1,spacer,camera_actions_2])
actions=VBox([tab,HBox([Label('Camera axis: '),handler.button_x,handler.button_y,handler.button_z,helper_text])])

# player = Play(interval=50)


layout = Layout(display='flex',
                flex_flow='column',
                align_items='center',)
# slider = IntSlider(max=50)
# jslink((player, 'value'), (slider, 'value'))
# jslink((player, 'value'), (handler.view, 'frame'))
# def on_display(w):
#     c = """

#     var pe = document.getElementsByClassName('what')[0]

#     pe.style.position = 'absolute'
#     pe.style.zIndex = 100
#     pe.style.top = '10%'
#     pe.style.left = '10%'
#     this.$container.append($(pe))
#     this.$player.hide();
#     """
#     view._execute_js_code(c)

# box = HBox([player,slider])
# box.add_class('what')
# box.on_displayed(on_display)

In [ ]:
%%html
<style>
.box_style{
    border : 2px solid red;
}
</style>

In [ ]:
display(actions,HBox([handler.view],layout=layout).add_class("box_style"))


<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

# Using the interactive visualization


### Phonon viewer
The bottom view shows an interactive animation of a lattice vibration.

- Hover over the view to access the play and pause buttons.

- Rotate the surface by left clicking and dragging.

- Translate the surface by right clicking and dragging.

You can reset the view by clicking the corresponding 'camera axis' buttons.

A specific camera view can be given in the 'Advanced' tab.

### Phonon
Under the 'Phonon' tab are parameters related to the atomic chain and the phonon dispersion plot.

Click on the phonon dispersion plot to get the corresponding lattice vibration.


### Appearance
Under the 'Appearance' tab are parameters related to showing arrows along the oscillations or modifying the atoms appearance.


### Camera settings
Under the 'Camera settings' tab, a custom camera position can be sent to the viewer.


### Generate GIF
A GIF of the actual animation can be generated and downloaded.

Different rendering resolutions can be selected for convenience.
The animation speed will be reflected in the GIF animation speed.

Clicking 'Render GIF' will start savings each frames of the animation and then compiling them into a GIF.
During the GIF rendering, the view will flicker.
Do not change browser window or the rendering will fail.

Once the rendering has been successfully done, a preview of the GIF is shown. Right click on it to download the GIF.
